<img src='imgs/cassandra.svg' width='750'/>

# docker

In [32]:
!docker run -d                   \
    -v ${PWD}/cassandra_vol:/ddl \
    -h cassandra                 \
    -p 7000:7000                 \
    -p 127.0.0.1:9042:9042       \
    -p 127.0.0.1:9160:9160       \
    --name cassandra             \
    cassandra:3.11

d9d964c7b15de9d1dca03a73be3b99a0bb96d2ce300be4f863537e73d7004e48


# validate

In [33]:
!docker ps | grep cassandra

d9d964c7b15d        cassandra:3.11        "docker-entrypoint.s…"   4 seconds ago       Up 3 seconds        7001/tcp, 127.0.0.1:9042->9042/tcp, 0.0.0.0:7000->7000/tcp, 7199/tcp, 127.0.0.1:9160->9160/tcp   cassandra


In [34]:
!docker exec -w /ddl -it cassandra /usr/bin/cqlsh --version

cqlsh 5.0.1


# DDL

create keyspace

In [36]:
!docker exec -it cassandra cqlsh -e \
"CREATE KEYSPACE elk_c WITH replication = {'class': 'NetworkTopologyStrategy', 'datacenter1': '2'} AND durable_writes = true"

create table

In [37]:
!docker exec -it cassandra cqlsh -e \
"use elk_c;CREATE TABLE employee(id int PRIMARY KEY,name text,city text,salary varint,phone varint)"

In [38]:
!docker exec -it cassandra cqlsh -e 'use elk_c;describe tables;' 


employee



In [39]:
!docker exec -it cassandra cqlsh -e 'use elk_c; describe employee;' 


CREATE TABLE elk_c.employee (
    id int PRIMARY KEY,
    city text,
    name text,
    phone varint,
    salary varint
) WITH bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND dclocal_read_repair_chance = 0.1
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair_chance = 0.0
    AND speculative_retry = '99PERCENTILE';



# data

generate some random data

In [40]:
employees_csv = open('cassandra_vol/employees.csv', 'w')
# id int PRIMARY KEY,
# name text
# city text
# salary varint
# phone varint
employees_csv.write('1,Adam,NOLA,60,5551234\n')
employees_csv.write('2,Bob,LA,70,3331234\n')
employees_csv.write('3,Chuck,NYC,80,6661234\n')
employees_csv.write('4,Dave,BOS,90,9991234\n')
employees_csv.write('5,Earl,RAL,100,1111234\n')

employees_csv.flush()
employees_csv.close()
!cat cassandra_vol/employees.csv

1,Adam,NOLA,60,5551234
2,Bob,LA,70,3331234
3,Chuck,NYC,80,6661234
4,Dave,BOS,90,9991234
5,Earl,RAL,100,1111234


In [41]:
!time docker exec -it cassandra cqlsh -e \
"use elk_c;COPY employee(id,name,city,salary,phone) FROM '/ddl/employees.csv' WITH DELIMITER=','"

Using 9 child processes

Starting copy of elk_c.employee with columns [id, name, city, salary, phone].
Processed: 5 rows; Rate:      16 rows/s; Avg. rate:      16 rows/sProcessed: 5 rows; Rate:       8 rows/s; Avg. rate:      12 rows/s
5 rows imported from 1 files in 0.421 seconds (0 skipped).

real	0m0.949s
user	0m0.031s
sys	0m0.018s


# validate data

In [42]:
!time docker exec -it cassandra \
cqlsh --connect-timeout 999 --request-timeout 999 \
-e "use elk_c;select count(*) from employee"


 count
-------
     5

(1 rows)

Warnings :
Aggregation query used without partition key


real	0m0.552s
user	0m0.032s
sys	0m0.018s


In [43]:
!time docker exec -it cassandra cqlsh -e "use elk_c;select * from employee limit 5"


 id | city | name  | phone   | salary
----+------+-------+---------+--------
  5 |  RAL |  Earl | 1111234 |    100
  1 | NOLA |  Adam | 5551234 |     60
  2 |   LA |   Bob | 3331234 |     70
  4 |  BOS |  Dave | 9991234 |     90
  3 |  NYC | Chuck | 6661234 |     80

(5 rows)

real	0m0.567s
user	0m0.031s
sys	0m0.020s


# stop & kill

In [30]:
#!docker stop cassandra && docker rm cassandra
# or restart
# !docker restart cassandra

cassandra
cassandra
